In [1]:
#Note for psplt.plot_complex_density_matrix() to properly plot you will need to comment out the following from pulsee.plot source code: 
#idx, = np.where(abs(dm_data) < 0.001)
#dm_data[idx] = abs(dm_data[idx])

In [2]:
import numpy as np
import qutip as qt
import pulsee.plot as psplt
import operators as op # "import *" is bad Python practice, but simplest way to do some math
from sympy import symbols, Matrix, nsimplify


# import pulsee.simulation as sim
from plot_dm import plot_complex_density_matrix

import tomography_helper_functions
import tomography_helper_functions as helper
# from tomography.tomography_helper_functions import projection_fortunato, plot_spectra_together
# from tomography.tomography_helper_functions import index_to_element, integrate_simpson, integrate_optimized

# import * is bad practice, but this is the easiest way to do quick theoretical calculations with this many variables.
import operators as op
import global_constants as glob
from coeff_groups_class import CoefficientGroups

from temporal_average import temporal_average

# Calculating thermal state density matrix

In [3]:
# Define symbolic constants
hbar, w1, w2, k_b, T, = symbols('hbar w1 w2 k_b T')

# Extract the numerical matrix from the Qobj as a NumPy array
op.Iz_numpy = op.Iz.full()
op.Sz_numpy = op.Sz.full()
op.identity_numpy = op.IDENTITY.full()
# Convert the NumPy array to a SymPy Matrix
op.Iz_sympy = Matrix(op.Iz_numpy)
op.Sz_sympy = Matrix(op.Sz_numpy)
op.identity_sympy = Matrix(op.identity_numpy)

thermal_density_matrix=(1/4)*op.identity_sympy+(1/4)*op.Iz_sympy*(hbar*w1/(k_b*T))+(1/4)*op.Sz_sympy*(hbar*w2/(k_b*T))
thermal_density_matrix_simple = thermal_density_matrix.applyfunc(nsimplify)
#spin dynamics pg 391
#thermal_state = thermal_state.applyfunc(Rational)
display(thermal_density_matrix_simple)
#qutip_matrix = Qobj(numerical_matrix)   # Create the Qobj from the numerical matrix

# Display the Qobj matrix
#print("The resulting Qobj matrix is:")
#print(qutip_matrix)

Matrix([
[1/4 + hbar*w1/(8*T*k_b) + hbar*w2/(8*T*k_b),                                           0,                                           0,                                           0],
[                                          0, 1/4 + hbar*w1/(8*T*k_b) - hbar*w2/(8*T*k_b),                                           0,                                           0],
[                                          0,                                           0, 1/4 - hbar*w1/(8*T*k_b) + hbar*w2/(8*T*k_b),                                           0],
[                                          0,                                           0,                                           0, 1/4 - hbar*w1/(8*T*k_b) - hbar*w2/(8*T*k_b)]])

In [4]:
# 3) Collect all variables in a list or set:
all_vars = [a_1, b_1, a_2, b_2, a_3, b_3, a_4, b_4]

# 4) Filter out any constrained variables:
free_vars = [v for v in all_vars if v not in constraint]

print("All variables:", all_vars)
print("Constraint: a_1 = 1")
print("Free variables:", free_vars)
print("Number of unknown (free) variables left:", len(free_vars))

NameError: name 'a_1' is not defined

In [ ]:
import sympy as sp

# 1) Define symbolic variables
a_1, b_1, a_2, b_2 = sp.symbols('a_1 b_1 a_2 b_2', complex=False)
a_3, b_3, a_4, b_4 = sp.symbols('a_3 b_3 a_4 b_4', complex=False)

# 2) Create your constraint a_1 = 1 as a symbolic Equation
equation = sp.Eq(abs(a_1+1j*b_1)**2 + abs(a_2+1j*b_2)**2 + abs(a_3+1j*b_3)**2 + abs(a_4+1j*b_4)**2, 1)

# 3) Define kets
ket_psi = sp.Matrix([
    [a_1 + 1j*b_1],
    [a_2 + 1j*b_2]
])

ket_psi_2 = sp.Matrix([
    [a_3 + 1j*b_3],
    [a_4 + 1j*b_4]
])

# Tensor product of the two kets
ket_psi_tensor_ket_psi_2 = sp.kronecker_product(ket_psi, ket_psi_2)
print("ket_psi_tensor_ket_psi_2 =")
print(ket_psi_tensor_ket_psi_2)

# Bra is the Hermitian transpose of the ket
bra_ket_psi_tensor_ket_psi_2 = ket_psi_tensor_ket_psi_2.H

# Operator |psi1><psi2| = ket * bra
op = ket_psi_tensor_ket_psi_2 * bra_ket_psi_tensor_ket_psi_2
print("\nOperator op =")
print(op)

# 4) Apply the constraint a_1 = 1
#    Option A: Direct substitution
op_constrained = op.subs(a_1+a_2+a_3+a_4+b_1+b_2+b_3+b_4, 1)

print("\nOperator op after substituting constraints")
print(op_constrained)


In [ ]:
solve op=M for the symby variables 

where


M = sp.Matrix([
    [7/24,  0,  0,  1/12],
    [0, 5/24,  0,  0],
    [0,  0,  5/24,  0],
    [1/12,  0,  0,  7/24]
])


In [ ]:

# 5) Example of a separate bra <phi|
x, y = sp.symbols('x y', complex=True)
bra_phi = sp.Matrix([[x, y]])  # row vector
print("\nBra <phi| =")
print(bra_phi)

# Outer product <phi|psi>
phi_psi = ket_psi * bra_phi  # yields a 1x1 matrix (scalar), i.e. <phi|psi>
print("\n<phi|psi> =")
display(phi_psi)

# Or an operator from (some) ket_phi * bra_phi:
# If you had a 'ket_phi' and did 'ket_phi * bra_phi',
# you'd get a 2x2 matrix in a 2D Hilbert space.


In [ ]:
import sympy as sp

# 1) Define symbolic variables for unknowns:
a1, b1, a2, b2, a3, b3, a4, b4 = sp.symbols('a1 b1 a2 b2 a3 b3 a4 b4', real=True)

# 2) Define your 8x8 matrix (EXAMPLE, not your real system):

# 3) Define the RHS vector (8x1):
c = sp.Matrix([1, 2, 3, 4, 5, 6, 7, 8])

# 4) Define the vector of unknowns:
x = sp.Matrix([a1, b1, a2, b2, a3, b3, a4, b4])

# 5) Solve M*x = c using LUsolve or gauss_jordan_solve:
solution_vector = M.LUsolve(c)
print("Solution vector (a1, b1, a2, b2, a3, b3, a4, b4) =")
print(solution_vector)

# Alternatively, solve using sp.solve:
# eqs = list(M*x - c)
# solutions = sp.solve(eqs, [a1, b1, a2, b2, a3, b3, a4, b4], dict=True)
# print(solutions)


In [13]:
import sympy as sp

# 1) Declare variables (real-valued)
a_1, b_1, a_2, b_2 = sp.symbols('a_1 b_1 a_2 b_2', real=True)
a_3, b_3, a_4, b_4 = sp.symbols('a_3 b_3 a_4 b_4', real=True)

# 2) Nonlinear constraint: sum of squares = 1
constraint_eq = sp.Eq(abs(a_1+1j*b_1)**2 + abs(a_2+1j*b_2)**2 + abs(a_3+1j*b_3)**2 + abs(a_4+1j*b_4)**2, 1)

# 3) Build kets and operator
ket_psi = sp.Matrix([[a_1 + 1j*b_1],
                     [a_2 + 1j*b_2]])
ket_psi_2 = sp.Matrix([[a_3 + 1j*b_3],
                       [a_4 + 1j*b_4]])

ket_psi_tensor_ket_psi_2 = sp.kronecker_product(ket_psi, ket_psi_2)
bra_psi_tensor_psi_2 = ket_psi_tensor_ket_psi_2.H
op = ket_psi_tensor_ket_psi_2 * bra_psi_tensor_psi_2

print("\nOriginal operator op (depends on 8 variables):")
print(op)

# 4) Solve the constraint for b_4 (as an example)
solutions = sp.solve(constraint_eq, b_4, dict=True)
print("\nSolutions for b_4:")
print(solutions)

# Pick the + sqrt(...) branch 
#THIS ONLY DOES POSITIVE BRANCH
b4_expr = solutions[0][b_4]
print("Chosen expression for b_4:", b4_expr)

# 5) Substitute back into the operator
op_sub = op.subs(b_4, b4_expr)
print("\nOperator op_sub after applying the constraint:")
print(op_sub)

# 6) Check which symbols remain
remaining_symbols = op_sub.free_symbols
print("\nSymbols that remain in op_sub:")
print(remaining_symbols)
print("Number of free variables:", len(remaining_symbols))


Original operator op (depends on 8 variables):
Matrix([[(a_1 - 1.0*I*b_1)*(a_1 + 1.0*I*b_1)*(a_3 - 1.0*I*b_3)*(a_3 + 1.0*I*b_3), (a_1 - 1.0*I*b_1)*(a_1 + 1.0*I*b_1)*(a_3 + 1.0*I*b_3)*(a_4 - 1.0*I*b_4), (a_1 + 1.0*I*b_1)*(a_2 - 1.0*I*b_2)*(a_3 - 1.0*I*b_3)*(a_3 + 1.0*I*b_3), (a_1 + 1.0*I*b_1)*(a_2 - 1.0*I*b_2)*(a_3 + 1.0*I*b_3)*(a_4 - 1.0*I*b_4)], [(a_1 - 1.0*I*b_1)*(a_1 + 1.0*I*b_1)*(a_3 - 1.0*I*b_3)*(a_4 + 1.0*I*b_4), (a_1 - 1.0*I*b_1)*(a_1 + 1.0*I*b_1)*(a_4 - 1.0*I*b_4)*(a_4 + 1.0*I*b_4), (a_1 + 1.0*I*b_1)*(a_2 - 1.0*I*b_2)*(a_3 - 1.0*I*b_3)*(a_4 + 1.0*I*b_4), (a_1 + 1.0*I*b_1)*(a_2 - 1.0*I*b_2)*(a_4 - 1.0*I*b_4)*(a_4 + 1.0*I*b_4)], [(a_1 - 1.0*I*b_1)*(a_2 + 1.0*I*b_2)*(a_3 - 1.0*I*b_3)*(a_3 + 1.0*I*b_3), (a_1 - 1.0*I*b_1)*(a_2 + 1.0*I*b_2)*(a_3 + 1.0*I*b_3)*(a_4 - 1.0*I*b_4), (a_2 - 1.0*I*b_2)*(a_2 + 1.0*I*b_2)*(a_3 - 1.0*I*b_3)*(a_3 + 1.0*I*b_3), (a_2 - 1.0*I*b_2)*(a_2 + 1.0*I*b_2)*(a_3 + 1.0*I*b_3)*(a_4 - 1.0*I*b_4)], [(a_1 - 1.0*I*b_1)*(a_2 + 1.0*I*b_2)*(a_3 - 1.0*I*b_3)*(a_4 + 

In [19]:
epsilon=0

# --- Step 2b: The target matrix M ---
M = sp.Matrix([
    [sp.Rational(1+epsilon,4), 0,               0,               sp.Rational(epsilon,2)],
    [0,                 sp.Rational(1-epsilon,4), 0,             0               ],
    [0,                 0,               sp.Rational(1-epsilon,4), 0             ],
    [sp.Rational(epsilon,2), 0,               0,               sp.Rational(1+epsilon,4)]
])

# --- Step 3: Equations for op = M ---
equations = []
for i in range(4):
    for j in range(4):
        equations.append(sp.Eq(op[i, j], M[i, j]))

# --- Step 4: Solve the system ---
vars_list = [a_1, b_1, a_2, b_2, a_3, b_3, a_4, b_4]
solutions = sp.solve(equations, vars_list, dict=True)

print("Number of solutions found:", len(solutions))
for i, sol in enumerate(solutions):
    print(f"\nSolution {i}:")
    for var, val in sol.items():
        print(f"  {var} = {val}")

# --- Step 5: (Optional) verify solution, if any ---
if solutions:
    sol_0 = solutions[0]
    op_sub = op.subs(sol_0)
    diff = op_sub - M
    print("\nCheck op - M for solution 0 (should be all 0s if exact):")
    print(sp.simplify(diff))


Number of solutions found: 0


In [21]:
import sympy as sp

# 1) Define the 4x4 matrix M (a density matrix) symbolically or numerically
#    Below is just an example. Replace with your actual M.
#    Make sure M is Hermitian, positive semidefinite, and trace=1 for a proper density matrix.
A = sp.Rational(1,4)
B = sp.Rational(0)
M = sp.Matrix([
    [A, 0,  0,  B ],
    [0, A,  B,  0 ],
    [0, B,  A,  0 ],
    [B, 0,  0,  A ]
])

# 2) Check basic properties (optional but good practice)
print("M is Hermitian?", M.is_hermitian)
print("Trace of M =", M.trace())

# 3) Compute eigenvalues & eigenvectors
#    eigenvects() returns a list of (eigenvalue, multiplicity, [eigenvectors...])
eigen_data = M.eigenvects()

print("\nEigen Decomposition of M:")
for (val, multiplicity, evec_list) in eigen_data:
    print(f"Eigenvalue = {val}, multiplicity = {multiplicity}")
    for evec in evec_list:
        print(f"  Eigenvector = {evec}")

# 4) Construct the spectral decomposition
#    We store the rank-1 projectors (|psi_k><psi_k|) scaled by eigenvalue lambda_k.
#    Note: Each evec should be normalized for the standard form lambda_k |psi_k><psi_k|.
spectral_terms = []
for (val, _mult, evec_list) in eigen_data:
    for evec in evec_list:
        # Normalize the eigenvector:
        norm_evec = evec / evec.norm()  # ensure length 1
        # Outer product:
        projector = norm_evec * norm_evec.H  # 4x1 times 1x4 => 4x4 matrix
        # Weighted by eigenvalue 'val':
        spectral_terms.append(val * projector)

# Sum the rank-1 projectors:
M_spectral = sum(spectral_terms)

# 5) Simplify or compare with original M
M_diff = sp.simplify(M_spectral - M)
print("\nCheck M_spectral - M (should be zero matrix):")
print(M_diff)

# 6) Print or manipulate final decomposition
print("\nSpectral Decomposition of M:\n")
for (val, _mult, evec_list) in eigen_data:
    for evec in evec_list:
        norm_evec = evec / evec.norm()
        print(f"Eigenvalue: {val}")
        print(f"Normalized eigenvector: {norm_evec}")
        print()


M is Hermitian? True
Trace of M = 1

Eigen Decomposition of M:
Eigenvalue = 1/4, multiplicity = 4
  Eigenvector = Matrix([[1], [0], [0], [0]])
  Eigenvector = Matrix([[0], [1], [0], [0]])
  Eigenvector = Matrix([[0], [0], [1], [0]])
  Eigenvector = Matrix([[0], [0], [0], [1]])


TypeError: unsupported operand type(s) for +: 'MutableDenseMatrix' and 'int'

In [1]:
import sympy as sp

def is_positive_semidefinite(mat):
    """Check if all eigenvalues of 'mat' are >= 0 (within numerical tolerance)."""
    vals = mat.eigenvalues()
    # We use a small tolerance for floating checks if it's symbolic-floating
    # If purely symbolic, one might do more advanced checks.
    for v in vals:
        if v.is_real is False:
            return False  # complex eigenvalue => not PSD
        if v.evalf() < -1e-14:
            return False  # negative eigenvalue => not PSD
    return True

def partial_transpose_2x2(mat):
    """
    Partial transpose of a 4x4 matrix 'mat' w.r.t. the second qubit.

    The basis is assumed: |00>, |01>, |10>, |11>.
    We'll reorder entries: row = (i1,i2), col = (j1,j2).
    Then swap i2 <-> j2 in the indexing.
    """
    if mat.shape != (4,4):
        raise ValueError("Matrix must be 4x4 for a 2-qubit partial transpose.")
    # We'll build a new 4x4 matrix PT = M^{T_2}
    PT = sp.zeros(4, 4)
    # Map basis index to (i1, i2):
    #  0 -> (0,0)
    #  1 -> (0,1)
    #  2 -> (1,0)
    #  3 -> (1,1)
    def idx_to_pair(idx):
        return (idx//2, idx % 2)

    def pair_to_idx(i1, i2):
        return 2*i1 + i2

    for r in range(4):
        i1, i2 = idx_to_pair(r)
        for c in range(4):
            j1, j2 = idx_to_pair(c)
            # Partial transpose => swap i2 <-> j2
            # new row = (i1, j2), new col = (j1, i2)
            new_r = pair_to_idx(i1, j2)
            new_c = pair_to_idx(j1, i2)
            PT[new_r, new_c] = mat[r, c]
    return PT

def is_separable_2qubit(M):
    """
    Check if a 4x4 matrix M (density matrix) is separable (2-qubit),
    using the PPT criterion, which is necessary & sufficient in 2x2.

    Returns True/False and partial transpose matrix if desired.
    """
    # Optional: check M is Hermitian, has trace=1, etc. We'll skip here for brevity.
    # 1) build partial transpose
    M_pt = partial_transpose_2x2(M)
    # 2) check positivity
    if not is_positive_semidefinite(M_pt):
        return False
    return True

In [2]:
# Suppose we have a 4x4 Sympy Matrix M
M = sp.eye(4)/4  # maximally mixed state

sep = is_separable_2qubit(M)
print("Is M separable?", sep)


AttributeError: 'MutableDenseMatrix' object has no attribute 'eigenvalues'

In [3]:
import cvxpy as cp
import numpy as np

def qubit_ket(alpha, beta):
    """
    Build a normalized 2D complex vector from real angles alpha, beta.
    A standard parameterization:
      |psi> = ( cos(alpha), e^{i beta} sin(alpha) )^T
    """
    return np.array([
        [np.cos(alpha)],
        [np.exp(1j*beta)*np.sin(alpha)]
    ], dtype=complex)

def product_projector(alpha, beta, gamma, delta):
    """
    Build the 4x4 matrix for 
      |psi> x |phi> <psi| x <phi|,
    given angles (alpha,beta) for |psi> 
    and (gamma,delta) for |phi>.
    """
    ket_psi = qubit_ket(alpha, beta)
    ket_phi = qubit_ket(gamma, delta)
    # Tensor product => 4x1
    ket_2qubit = np.kron(ket_psi, ket_phi)
    # Outer product => 4x4
    return ket_2qubit @ ket_2qubit.conj().T

def find_separable_decomposition(M, K=4, max_tries=10):
    """
    Try to find p_k >= 0, sum_k p_k=1, plus angles (alpha_k,beta_k,gamma_k,delta_k),
    s.t. sum_k p_k * P_k = M, with P_k rank-1 product projectors.
    
    We'll do a random-restart approach, not guaranteed to converge in all cases.
    """
    # We'll treat the real part & imag part of the matrix equality constraints
    M_np = np.array(M.tolist(), dtype=complex)  # convert Sympy Matrix -> NumPy
    n = 4

    best_sol = None
    best_error = 1e9

    for attempt in range(max_tries):
        # p_k as a CVXPY variable
        p_var = cp.Variable(K, nonneg=True)
        # angles as unconstrained real variables => we do random tries.
        # We'll do a "sample angles" approach, then build the sum manually
        # then define constraints for the matrix = M is complicated as is.
        # Instead, we'll do a direct numeric "least-squares" approach
        # to see if we can get close. A true solver would need more advanced methods.

        # For each k, random angles
        alphas = np.random.uniform(0, np.pi, K)
        betas  = np.random.uniform(0, 2*np.pi, K)
        gammas = np.random.uniform(0, np.pi, K)
        deltas = np.random.uniform(0, 2*np.pi, K)

        # We'll build sum_k p_k * P_k
        # But p_k are unknown. We'll guess angles for now,
        # and let p_k be found by linear constraints. That at least is linear in p_k.
        projs = []
        for k in range(K):
            Pk = product_projector(alphas[k], betas[k], gammas[k], deltas[k])
            projs.append(Pk)

        # Real part constraints: sum_k p_k Re(Pk) = Re(M)
        # Imag part constraints: sum_k p_k Im(Pk) = Im(M)
        # and sum_k p_k = 1
        # We'll flatten them into vectors for CVXPY's linear constraints

        # The dimension of each Pk is 4x4 => 16 real constraints for real part, 16 for imag part
        # We'll do:
        # sum_k p_k * Re(Pk[i,j]) = Re(M[i,j]) for all i,j
        # sum_k p_k * Im(Pk[i,j]) = Im(M[i,j]) for all i,j
        # In 2-qubit real physical density matrix M, typically Im(M) = 0 for many cases,
        # but let's do it fully.

        real_mat = np.real(M_np).flatten()
        imag_mat = np.imag(M_np).flatten()
        real_proj = [np.real(Pk).flatten() for Pk in projs]
        imag_proj = [np.imag(Pk).flatten() for Pk in projs]

        # We'll create two constraints:
        # sum_{k} p_k * real_proj[k] = real_mat
        # sum_{k} p_k * imag_proj[k] = imag_mat
        # sum_k p_k = 1
        constraints = []
        # for each index idx in 0..15, sum_{k} p_k * real_proj[k][idx] = real_mat[idx]
        for idx in range(n*n):
            row_expr_real = 0
            row_expr_imag = 0
            for k in range(K):
                row_expr_real += p_var[k] * real_proj[k][idx]
                row_expr_imag += p_var[k] * imag_proj[k][idx]
            constraints.append(row_expr_real == real_mat[idx])
            constraints.append(row_expr_imag == imag_mat[idx])

        # Probability simplex
        constraints.append(cp.sum(p_var) == 1)

        # We set up a "dummy" objective => minimize(0).
        # So we do a feasibility problem: find any p_var satisfying constraints
        prob = cp.Problem(cp.Minimize(0), constraints)

        try:
            result = prob.solve(solver=cp.SCS, verbose=False)
        except:
            continue

        if prob.status in ["optimal", "optimal_inaccurate"]:
            # Check how well we match
            residual = prob.value
            # We'll build the final decomposition
            p_sol = p_var.value
            # Reconstruct sum_k p_k Pk
            test_mat = sum( p_sol[k] * projs[k] for k in range(K) )
            err = np.linalg.norm(test_mat - M_np, ord='fro')
            if err < best_error:
                best_error = err
                best_sol = (p_sol, (alphas, betas, gammas, deltas))
            if best_error < 1e-12:
                break  # good enough

    if best_sol is None:
        return None
    # Return final best decomposition
    p_sol, angles_sol = best_sol
    alphas, betas, gammas, deltas = angles_sol
    return p_sol, alphas, betas, gammas, deltas, best_error

# Usage example
if __name__ == "__main__":
    import sympy
    # Build an example 2-qubit separable state:
    # M = 0.5 * |00><00| + 0.5 * |10><10|
    # in 2-qubit space => diag(0.5, 0, 0.5, 0)
    M_example = sympy.zeros(4,4)
    M_example[0,0] = 0.5
    M_example[2,2] = 0.5

    # Check PPT
    if not is_separable_2qubit(M_example):
        print("Matrix is entangled or not PPT => no decomposition.")
    else:
        print("Matrix passes PPT => it is separable or borderline.")
        # Attempt a numeric decomposition with K=2 or K=4
        res = find_separable_decomposition(M_example, K=2, max_tries=20)
        if res is None:
            print("No decomposition found with this approach.")
        else:
            p_sol, alphas, betas, gammas, deltas, best_err = res
            print("\nFound a decomposition with final Fro norm error:", best_err)
            for k in range(len(p_sol)):
                print(f"k={k}, p={p_sol[k]} angles = (alpha={alphas[k]}, beta={betas[k]}, gamma={gammas[k]}, delta={deltas[k]})")


ModuleNotFoundError: No module named 'cvxpy'